In [1]:
import pandas as pd
import numpy as np
from datetime import timedelta
from features import truncated_mean, calculate_conversion_rate

In [2]:
# Dataset 1: User demographics
customer_data = pd.read_csv('../data/raw/user_demographics.csv', parse_dates=['reg_date'], infer_datetime_format=True)
customer_data.uid = customer_data.uid.astype('int')
customer_data.age = customer_data.age.astype('int')
customer_data.head()

,uid,reg_date,device,gender,country,age
0,54030035,2017-06-29,and,M,USA,19
1,72574201,2018-03-05,iOS,F,TUR,22
2,64187558,2016-02-07,iOS,M,USA,16
3,92513925,2017-05-25,and,M,BRA,41
4,99231338,2017-03-26,iOS,M,FRA,59


In [3]:
customer_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   uid       10000 non-null  int64         
 1   reg_date  10000 non-null  datetime64[ns]
 2   device    10000 non-null  object        
 3   gender    10000 non-null  object        
 4   country   10000 non-null  object        
 5   age       10000 non-null  int64         
dtypes: datetime64[ns](1), int64(2), object(3)
memory usage: 468.9+ KB


In [4]:
# Dataset 2: User actions
app_purchases = pd.read_csv('../data/raw/inapp_purchases.csv', parse_dates=['date'], infer_datetime_format=True)
app_purchases.head()


,date,uid,sku,price
0,2017-07-10,41195147,sku_three_499,499
1,2017-07-15,41195147,sku_three_499,499
2,2017-11-12,41195147,sku_four_599,599
3,2017-09-26,91591874,sku_two_299,299
4,2017-12-01,91591874,sku_four_599,599


In [5]:
app_purchases.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9006 entries, 0 to 9005
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   date    9006 non-null   datetime64[ns]
 1   uid     9006 non-null   int64         
 2   sku     9006 non-null   object        
 3   price   9006 non-null   int64         
dtypes: datetime64[ns](1), int64(2), object(1)
memory usage: 281.6+ KB


In [6]:
# Merge on the 'uid' field
purchase_data = app_purchases.merge(customer_data, on=['uid'], how='inner')
# Examine the results 
print(purchase_data.head())
print(len(purchase_data))
print(purchase_data.info())
purchase_data.to_csv('../data/clean/purchase_data.csv', index=False)

        date       uid            sku  price   reg_date device gender country  \
0 2017-07-10  41195147  sku_three_499    499 2017-06-26    and      M     BRA   
1 2017-07-15  41195147  sku_three_499    499 2017-06-26    and      M     BRA   
2 2017-11-12  41195147   sku_four_599    599 2017-06-26    and      M     BRA   
3 2017-09-26  91591874    sku_two_299    299 2017-01-05    and      M     TUR   
4 2017-12-01  91591874   sku_four_599    599 2017-01-05    and      M     TUR   

   age  
0   17  
1   17  
2   17  
3   17  
4   17  
9006
<class 'pandas.core.frame.DataFrame'>
Int64Index: 9006 entries, 0 to 9005
Data columns (total 9 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   date      9006 non-null   datetime64[ns]
 1   uid       9006 non-null   int64         
 2   sku       9006 non-null   object        
 3   price     9006 non-null   int64         
 4   reg_date  9006 non-null   datetime64[ns]
 5   device    9006 non-nu

In [7]:
purchase_data.groupby(by=['country','device'], axis=0, as_index=False).agg({'price':np.mean, 'age':np.mean})

,country,device,price,age
0,BRA,and,412.985594,23.913565
1,BRA,iOS,404.739300,23.975681
2,CAN,and,406.826087,23.282609
3,CAN,iOS,386.573964,25.023669
4,DEU,and,402.474903,23.814672
5,DEU,iOS,417.639798,20.773300
6,FRA,and,418.377163,23.200692
7,FRA,iOS,382.921569,24.407843
8,TUR,and,433.913793,24.004310
9,TUR,iOS,390.176471,22.591176


In [8]:

purchase_data.groupby(by=['device', 'country'], axis=0, as_index=False).agg({'age':truncated_mean})

,device,country,age
0,and,BRA,22.798977
1,and,CAN,23.335000
2,and,DEU,21.640257
3,and,FRA,21.786885
4,and,TUR,22.422572
5,and,USA,22.241203
6,iOS,BRA,22.784946
7,iOS,CAN,24.048780
8,iOS,DEU,19.611413
9,iOS,FRA,23.123457


In [9]:
print(type(purchase_data.groupby(by=['device', 'country'], axis=0, as_index=False)))

<class 'pandas.core.groupby.generic.DataFrameGroupBy'>


In [10]:
# Calculate the mean purchase price 
purchase_price_mean = purchase_data.agg({'price': ['mean', 'median'], 'age': ['mean', 'median']})

# Examine the output 
print(purchase_price_mean)


             price        age
mean    406.772596  23.922274
median  299.000000  21.000000


In [11]:
# Group the data 
grouped_purchase_data = purchase_data.groupby(by=['device', 'gender'])

# Aggregate the data
purchase_summary = grouped_purchase_data.agg({'price': ['mean', 'median', 'std']})

# Examine the results
print(purchase_summary)

                    price                   
                     mean median         std
device gender                               
and    F       400.747504  299.0  179.984378
       M       416.237308  499.0  195.001520
iOS    F       404.435330  299.0  181.524952
       M       405.272401  299.0  196.843197


In [12]:
purchase_data.head()

,date,uid,sku,price,reg_date,device,gender,country,age
0,2017-07-10,41195147,sku_three_499,499,2017-06-26,and,M,BRA,17
1,2017-07-15,41195147,sku_three_499,499,2017-06-26,and,M,BRA,17
2,2017-11-12,41195147,sku_four_599,599,2017-06-26,and,M,BRA,17
3,2017-09-26,91591874,sku_two_299,299,2017-01-05,and,M,TUR,17
4,2017-12-01,91591874,sku_four_599,599,2017-01-05,and,M,TUR,17


In [13]:
# conversion rate for 7 days
conversion_rate = calculate_conversion_rate(purchase_data, 7)
print(conversion_rate)

# conversion rate for 14 days
conversion_rate = calculate_conversion_rate(purchase_data, 14)
print(conversion_rate)

0.021473075211392967
0.03775896636221876


In [14]:
# Find days between subscription and registration if they
# subscribed ... and pd.NaT otherwise
sub_time = np.where(
    # if: subscription date exist
    purchase_data.date.notnull(),

    # then: how many days since they register
    (purchase_data.date - purchase_data.reg_date).dt.days,

    # else:
    pd.NaT
)

purchase_data['sub_time'] = sub_time

In [15]:
# Set the max registration date to be one month before today
current_date = purchase_data.reg_date.max()
max_reg_date = current_date - timedelta(days=28)

# Find the month 1 values
month1 = np.where((purchase_data.reg_date < max_reg_date) &
                 (purchase_data.date < purchase_data.reg_date + timedelta(days=28)),
                  purchase_data.price, 
                  np.NaN)
                 
# Update the value in the DataFrame
purchase_data['month1'] = month1

# Group the data by gender and device 
purchase_data_upd = purchase_data.groupby(by=['gender', 'device'], as_index=False) 

# Aggregate the month1 and price data 
purchase_summary = purchase_data_upd.agg(
                        {'month1': ['mean', 'median'],
                        'price': ['mean', 'median']})

# Examine the results 
print(purchase_summary)

  gender device      month1              price       
                       mean median        mean median
0      F    and  388.204545  299.0  400.747504  299.0
1      F    iOS  432.587786  499.0  404.435330  299.0
2      M    and  413.705882  399.0  416.237308  499.0
3      M    iOS  433.313725  499.0  405.272401  299.0
